In [1]:
# Create function to match label to letter
def getLetter(result):
    classLabels = { 0: 'A',
                    1: 'B',
                    2: 'C',
                    3: 'D',
                    4: 'E',
                    5: 'F',
                    6: 'G',
                    7: 'H',
                    8: 'I',
                    9: 'K',
                    10: 'L',
                    11: 'M',
                    12: 'N',
                    13: 'O',
                    14: 'P',
                    15: 'Q',
                    16: 'R',
                    17: 'S',
                    18: 'T',
                    19: 'U',
                    20: 'V',
                    21: 'W',
                    22: 'X',
                    23: 'Y',
                    24: ' '}
    try:
        res = int(result)
        return classLabels[res]
    except:
        return "Error"

In [2]:
def max(liste):
    c=0
    for i in range(len(liste)):
        if liste[i]>c:
            c=liste[i]
    return liste.index(c)

In [3]:
from tensorflow.keras.models import load_model
classifier= load_model('model_200_samples.h5')

In [4]:
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_hands   = mp.solutions.hands

In [6]:
import cv2 as cv2
import time
import gtts
import os
import numpy as np

time_delay = 15

ref = time.time()
cap = cv2.VideoCapture(0)
msg = ""

with mp_hands.Hands(min_detection_confidence=0.8,min_tracking_confidence=0.5) as hands:
    while True:

        ret, frame = cap.read()
        frame=cv2.flip(frame, 1)

        #define region of interest   
        #roi = frame[100:400, 320:620]
        roi = frame[100:350, 320:570]
        roi = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
        roi.flags.writeable= False
        results = hands.process(roi)
        roi.flags.writeable= True
        #zeros=np.zeros(roi.shape)
        if results.multi_hand_landmarks:
            for num,hand in enumerate(results.multi_hand_landmarks):
                zeros=np.zeros(roi.shape)
                mp_drawing.draw_landmarks(zeros,hand,mp_hands.HAND_CONNECTIONS)
                #cv2.imshow('hand',zeros)
                zeros = cv2.resize(zeros, (64, 64))
                cv2.imwrite("ok" + ".jpg", zeros)
                zeros=cv2.imread("ok.jpg",cv2.IMREAD_GRAYSCALE)
                zeros = zeros.reshape(1,64,64,1)
                zeros=zeros/255
                prediction=classifier.predict(zeros, 1, verbose = 0)[0]
                prediction=list(prediction)
                ma = max(prediction)
                cv2.putText(frame,getLetter(ma), (300 , 100), cv2.FONT_HERSHEY_COMPLEX, 2, (0, 255, 0), 2)

        cv2.rectangle(frame, (320, 100), (570, 350), (255,0,0), 5)
        cv2.imshow('frame', frame)

        #if time.time() >= ref + time_delay:
        if cv2.waitKey(10) == 13:   
            msg = msg + getLetter(ma)
            print(getLetter(ma),end="")

        if cv2.waitKey(10) == 32:#13 is the En ter Key
            break
    print() 
    cap.release()   
    cv2.destroyAllWindows()
    print("Your message is : ",msg)
    speech =gtts.gTTS(text=msg,lang='en')
    speech.save("audio.mp3")  
    os.system("audio.mp3")  
    cap.release()   
    cv2.destroyAllWindows()


Your message is :  


AssertionError: No text to speak